### Import

In [1]:
import torch
import torch.optim as optim
import numpy as np

In [2]:
# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

### Input 생성

In [3]:
sample = " if you want you"

### 데이터  가공

char단위로 중복제거 후 딕셔너리의 저장

In [4]:
# make dictionary
char_set = list(set(sample)) # set에 넣게 되면 중복이 제거 된다.
char_dic = {c: i for i, c in enumerate(char_set)}
print(char_dic)

{'u': 0, 'a': 1, 'w': 2, 'i': 3, 'o': 4, ' ': 5, 't': 6, 'f': 7, 'y': 8, 'n': 9}


In [5]:
# hyper parameters
dic_size = len(char_dic) # input size
hidden_size = len(char_dic) # hidden state size
learning_rate = 0.1

Index로 변환 (one-hot)

In [6]:
# data setting
sample_idx = [char_dic[c] for c in sample]
x_data = [sample_idx[:-1]]
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
y_data = [sample_idx[1:]]

sample_idx 확인

In [7]:
char_dic

{'u': 0,
 'a': 1,
 'w': 2,
 'i': 3,
 'o': 4,
 ' ': 5,
 't': 6,
 'f': 7,
 'y': 8,
 'n': 9}

In [8]:
sample_idx

[5, 3, 7, 5, 8, 4, 0, 5, 2, 1, 9, 6, 5, 8, 4, 0]

x_data, y_data 확인

In [9]:
x_data

[[5, 3, 7, 5, 8, 4, 0, 5, 2, 1, 9, 6, 5, 8, 4]]

In [10]:
y_data

[[3, 7, 5, 8, 4, 0, 5, 2, 1, 9, 6, 5, 8, 4, 0]]

만든 데이터 tensor로 변경

In [11]:
# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

/var/folders/vc/qmhd_frs3436xpj2_4dshbtw0000gn/T/ipykernel_57393/1455435457.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/miniforge3/conda-bld/pytorch-recipe_1643987637853/work/torch/csrc/utils/tensor_new.cpp:201.)
  X = torch.FloatTensor(x_one_hot)


### 모델 선언

In [12]:
# declare RNN
rnn = torch.nn.RNN(dic_size, hidden_size, batch_first=True) # batch dimention이 먼저 오게 변경

### Loss function & optimizer

In [13]:
# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss() # loss function 
optimizer = optim.Adam(rnn.parameters(), learning_rate) # optimizer

argmax()
- https://gomguard.tistory.com/145

In [15]:
# start training
for i in range(50):
    optimizer.zero_grad()
    outputs, _status = rnn(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1)) # (모델의 output, 정답 label)
    loss.backward()
    optimizer.step()

    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  0.8249279856681824 prediction:  [[3 7 5 8 4 0 5 2 1 9 6 5 8 4 0]] true Y:  [[3, 7, 5, 8, 4, 0, 5, 2, 1, 9, 6, 5, 8, 4, 0]] prediction str:  if you want you
1 loss:  0.8246438503265381 prediction:  [[3 7 5 8 4 0 5 2 1 9 6 5 8 4 0]] true Y:  [[3, 7, 5, 8, 4, 0, 5, 2, 1, 9, 6, 5, 8, 4, 0]] prediction str:  if you want you
2 loss:  0.8240357637405396 prediction:  [[3 7 5 8 4 0 5 2 1 9 6 5 8 4 0]] true Y:  [[3, 7, 5, 8, 4, 0, 5, 2, 1, 9, 6, 5, 8, 4, 0]] prediction str:  if you want you
3 loss:  0.8234229683876038 prediction:  [[3 7 5 8 4 0 5 2 1 9 6 5 8 4 0]] true Y:  [[3, 7, 5, 8, 4, 0, 5, 2, 1, 9, 6, 5, 8, 4, 0]] prediction str:  if you want you
4 loss:  0.8231651782989502 prediction:  [[3 7 5 8 4 0 5 2 1 9 6 5 8 4 0]] true Y:  [[3, 7, 5, 8, 4, 0, 5, 2, 1, 9, 6, 5, 8, 4, 0]] prediction str:  if you want you
5 loss:  0.8226655125617981 prediction:  [[3 7 5 8 4 0 5 2 1 9 6 5 8 4 0]] true Y:  [[3, 7, 5, 8, 4, 0, 5, 2, 1, 9, 6, 5, 8, 4, 0]] prediction str:  if you want you
6 loss:  0